In [65]:
import numpy as np
import base64
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes

from sklearn.decomposition import NMF, LatentDirichletAllocation
from matplotlib import pyplot as plt
%matplotlib inline
from gensim import corpora, models, similarities
import random
# print "All reached here twice now...."



In [3]:
train = pd.read_csv("/home/shringi/Downloads/offline_challenge/offline_challenge/xtrain_obfuscated.txt")
print("Number of rows in train dataset : ",train.shape[0])
# print(train.shape)
# print (train)
# train.head()
# train.values.tolist()
lines = [line.rstrip('\n') for line in open("/home/shringi/Downloads/offline_challenge/offline_challenge/xtrain_obfuscated.txt")]
# with open("/home/dupree/Downloads/offline_challenge/offline_challenge/xtrain_obfuscated.txt") as f:
#     lines = f.readlines()
# random.shuffle(lines)
train_data_x = lines[:26010]
test_data_x = lines[26010:]

lines = [line.rstrip('\n') for line in open("/home/shringi/Downloads/offline_challenge/offline_challenge/ytrain.txt")]

# with open("/home/dupree/Downloads/offline_challenge/offline_challenge/ytrain.txt") as f:
#     lines = f.readlines()
# random.shuffle(lines)
train_data_y = lines[:26010]
test_data_y = lines[26010:]

# print len(train_data_x)
# print len(test_data_x)
# print len(train_data_y)
# print len(test_data_y)

print train_data_x[:3]
print train_data_y[:3]


('Number of rows in train dataset : ', 32512)
['satwamuluhqgulamlrmvezuhqvkrpmletwulcitwskuhlemvtwamuluhiwiwenuhlrvimvqvkruhulenamuluhqgqvtwvimviwuhtwamuluhulqvkrenamcitwuhvipmpmqvuhskiwkrpmdfuhlrvimvskvikrpmqvuhskmvgzenleuhqvmvamuluhulenamuluhqvletwtwvipmpmgzleenamuhtwamuluhtwletwdfuhiwkrxeleentwxeuhpmqvuhtwiwmvamdfuhpkeztwamuluhvimvuhqvtwmkpmpmlelruhgztwtwskuhtwlrkrpmlruhpmuluhqvenuhtwyplepmxeuhenuhamypkrqvuhamulmvdfuhqvskentwamletwlrlrpmiwuhtwamul', 'twmkiwpmqvtwleuhsaiwsktwmvlelekramuhqvkruhtwskenezuhskvienuhqgulvienulqvvimvuhvienuhvimvuhulyptwbrtvkrqvuhtwamuluhsktwlrvienamypuhqvmvamguuhvgoaulamlrmvvibhpmuluhqvijulmvnkuhqgskkrpmiwenuhsktwskskenuhsaiwmvleenulvikriwpmmkvimvuhiguhvgqgulleentwamuhsaezuhqvqvtwiwtvuhskvisknkuhravidfpmvitwleuhvienmvypqvpmjeuhxepmuhlekrtwulenezenuhiwenmvypvimvmkpmlegzuhsktwulenletvtwiwtwypuhtwamuluhpmul', 'vidfpmskuhvilepmuluhtwtvuhulsovienamqvuhskiwmvamypuhtwamulsouhqgvienezuhtwamuluhamulmvdfuhsaiwulvitwiwpmmvmkuhlrvimviwlrlrkrleulqvuhqgiwlemvlruhtwamuluh

In [7]:
### Fit transform the tfidf vectorizer ###
tfidf_vec = TfidfVectorizer(ngram_range=(1,5), analyzer='char')
full_tfidf = tfidf_vec.fit_transform(train_data_x + test_data_x)
train_tfidf = tfidf_vec.transform(train_data_x)
test_tfidf = tfidf_vec.transform(test_data_x)
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([26010, 3])




In [8]:
# print type(full_tfidf)
# print full_tfidf.get_shape()
# print train_tfidf.get_shape()
# print test_tfidf.get_shape()
kf = model_selection.KFold(n_splits=5, shuffle=False, random_state=None)

# for train_index, test_index in kf.split(train_data_x):
# #     print("TRAIN:", train_index, "TEST:", test_index)
#     print train_index, test_index
# #     X_train, X_test = train_data_x[train_index], train_data_x[test_index]
# #     y_train, y_test = train_data_y[train_index], train_data_y[test_index]
#     print len(train_index), len(test_index)

print train_tfidf.shape[0]

train_data_y = np.asarray(train_data_y)
for train_index, val_index in kf.split(train_data_x):
    train_X, val_X = train_tfidf[train_index], train_tfidf[val_index]
    train_y, val_y = train_data_y[train_index], train_data_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(train_X, train_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
#     pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print type(pred_test_y)
#     print "dev_X, val_X", dev_X, val_X 
#     print "dev_y, val_y", dev_y, val_y
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.
print pred_full_test

26010
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
('Mean cv score : ', 3.1276120042373847)
[[  1.19626796e-11   1.15195811e-02   7.93952199e-03 ...,   4.57994276e-01
    1.11978317e-02   2.63548983e-08]
 [  3.96079970e-11   2.75968045e-02   1.21315664e-02 ...,   2.18864084e-01
    3.50058437e-02   6.35578780e-08]
 [  2.98951027e-11   1.43037780e-01   1.24173516e-02 ...,   4.77800396e-01
    5.73977827e-02   1.34920468e-07]
 ..., 
 [  3.51506188e-11   1.70320447e-02   5.21268873e-02 ...,   4.42987899e-01
    2.35565616e-02   1.28158293e-07]
 [  2.05111007e-11   3.40224544e-02   1.16163012e-02 ...,   5.03024509e-01
    4.65865772e-02   7.52017707e-08]
 [  1.41125165e-11   4.58183303e-02   1.35308800e-02 ...,   2.05471229e-01
    2.77940948e-02   2.27954862e-08]]


In [66]:
train_data_y = lines[:26010]
test_data_y = lines[26010:]

print len(train_data_x)
print len(test_data_x)
print len(train_data_y)
print len(test_data_y)


# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='char',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(train_data_x) + list(test_data_x))
xtrain_tfv =  tfv.transform(train_data_x) 
xvalid_tfv = tfv.transform(test_data_x)
# print type(test_data_y)

26010
6503
26010
6503


In [8]:
def tokenize_chars(s):
        return list(s)
print tokenize_chars(train_data_x[0])

['s', 'a', 't', 'w', 'a', 'm', 'u', 'l', 'u', 'h', 'q', 'g', 'u', 'l', 'a', 'm', 'l', 'r', 'm', 'v', 'e', 'z', 'u', 'h', 'q', 'v', 'k', 'r', 'p', 'm', 'l', 'e', 't', 'w', 'u', 'l', 'c', 'i', 't', 'w', 's', 'k', 'u', 'h', 'l', 'e', 'm', 'v', 't', 'w', 'a', 'm', 'u', 'l', 'u', 'h', 'i', 'w', 'i', 'w', 'e', 'n', 'u', 'h', 'l', 'r', 'v', 'i', 'm', 'v', 'q', 'v', 'k', 'r', 'u', 'h', 'u', 'l', 'e', 'n', 'a', 'm', 'u', 'l', 'u', 'h', 'q', 'g', 'q', 'v', 't', 'w', 'v', 'i', 'm', 'v', 'i', 'w', 'u', 'h', 't', 'w', 'a', 'm', 'u', 'l', 'u', 'h', 'u', 'l', 'q', 'v', 'k', 'r', 'e', 'n', 'a', 'm', 'c', 'i', 't', 'w', 'u', 'h', 'v', 'i', 'p', 'm', 'p', 'm', 'q', 'v', 'u', 'h', 's', 'k', 'i', 'w', 'k', 'r', 'p', 'm', 'd', 'f', 'u', 'h', 'l', 'r', 'v', 'i', 'm', 'v', 's', 'k', 'v', 'i', 'k', 'r', 'p', 'm', 'q', 'v', 'u', 'h', 's', 'k', 'm', 'v', 'g', 'z', 'e', 'n', 'l', 'e', 'u', 'h', 'q', 'v', 'm', 'v', 'a', 'm', 'u', 'l', 'u', 'h', 'u', 'l', 'e', 'n', 'a', 'm', 'u', 'l', 'u', 'h', 'q', 'v', 'l', 'e',

In [10]:
# Fitting a simple Logistic Regression on TFIDF
from sklearn.linear_model import LogisticRegression

def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota


clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, train_data_y)
predictions = clf.predict_proba(xvalid_tfv)

print type(predictions)
print predictions.shape[0]


<type 'numpy.ndarray'>
6503


In [6]:
import math
print type(predictions)
print predictions.shape[0]
print predictions.shape[1]
print predictions[:4]

# metrics.log_loss(val_y, pred_val_y)
# print ("logloss: %0.3f " % metrics.accuracy_score(np.asarray(test_data_y), np.asarray(predictions.round())))

print ("logloss: %0.3f " % metrics.log_loss(test_data_y, predictions))
# print ("logloss: %0.3f " % metrics.mean_absolute_error(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"
# print str(math.exp(-(0.05))) + "% is the accuracy percentage"

<type 'numpy.ndarray'>
6503
12
[[ 0.0025959   0.03982532  0.02889083  0.02806657  0.05414063  0.63364265
   0.05985316  0.01514786  0.00914334  0.09789767  0.01413626  0.0166598 ]
 [ 0.00702954  0.05273785  0.13342085  0.10302957  0.004141    0.12929616
   0.00328851  0.02424679  0.46388523  0.00470743  0.06857282  0.00564426]
 [ 0.00289147  0.23618731  0.02724405  0.02272681  0.01831079  0.05084463
   0.09632029  0.29937573  0.01623429  0.00240868  0.10995325  0.11750272]
 [ 0.00868383  0.00205862  0.00340142  0.00973028  0.01390498  0.02012879
   0.03123459  0.84305834  0.00368125  0.01364261  0.01113381  0.03934149]]
logloss: 0.999 
0.368413520395% is the accuracy percentage


In [11]:
ctv = CountVectorizer(analyzer='char',
            ngram_range=(1, 3))

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(train_data_x + test_data_x)
xtrain_ctv =  ctv.transform(train_data_x) 
xvalid_ctv = ctv.transform(test_data_x)

# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, train_data_y)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"
print test_data_y[:15]
print predictions[:15]

logloss: 0.749 
0.473040735892% is the accuracy percentage
['3', '6', '5', '5', '5', '8', '6', '6', '3', '11', '9', '3', '0', '7', '4']
[[  2.97318159e-06   1.10897887e-04   4.23011597e-04   1.74918032e-03
    3.92525113e-03   9.76273714e-01   1.86638017e-04   6.25400868e-03
    1.68136332e-05   1.08714859e-02   9.04665465e-05   9.55589715e-05]
 [  3.15605121e-07   3.45642926e-03   6.41779425e-02   1.09031521e-02
    2.31007418e-10   4.40116894e-02   3.64839478e-12   3.65422701e-03
    8.30714295e-01   2.09564446e-10   4.30549106e-02   2.70379590e-05]
 [  6.79973697e-10   2.29253167e-04   6.39394003e-06   8.59652701e-07
    2.92258169e-06   6.26834645e-06   4.23104809e-02   5.49883469e-01
    1.01142506e-03   4.37261857e-08   4.06219068e-01   3.29814989e-04]
 [  5.19388973e-10   5.18946744e-13   1.36493151e-07   6.98428786e-12
    9.83708056e-07   1.62302814e-06   3.06163680e-03   9.96918831e-01
    1.02302257e-06   1.05287600e-06   3.01322150e-10   1.47118904e-05]
 [  5.25343703e-08  

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.svm import SVC
import math
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, train_data_y)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, train_data_y)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"


# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

print "Starting with fitting SVM"
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, train_data_y)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"

logloss: 1.473 
0.229172949824% is the accuracy percentage
logloss: 5.039 
0.00647973566916% is the accuracy percentage
Starting with fitting SVM
logloss: 1.087 
0.337154602202% is the accuracy percentage


In [13]:
# Fitting a simple xgboost on tf-idf
import xgboost as xgb
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), train_data_y)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print "Came till multiclass logloss funtion...."
print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"


/home/shringi/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



Came till multiclass logloss funtion....
logloss: 0.772 
0.461966812226% is the accuracy percentage


In [14]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, train_data_y)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"


logloss: 1.245 
0.287985656909% is the accuracy percentage


In [15]:
# Fitting a simple xgboost on tf-idf CountVectorizer features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), train_data_y)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"

logloss: 0.740 
0.477161737492% is the accuracy percentage


In [69]:
#Grid Searching on available models now
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

mll_scorer = metrics.make_scorer(metrics.log_loss, greater_is_better=False, needs_proba=True)

# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

logloss: 0.772 
0.461966812226% is the accuracy percentage


In [66]:
# Initialize Grid Search Model
from sklearn.model_selection import GridSearchCV
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, np.asarray(train_data_y))  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] lr__penalty=l1, lr__C=0.1, svd__n_components=120 ................
[CV] lr__penalty=l1, lr__C=0.1, svd__n_components=120 ................
[CV] lr__penalty=l1, lr__C=0.1, svd__n_components=180 ................
[CV] lr__penalty=l1, lr__C=0.1, svd__n_components=180 ................
[CV] lr__penalty=l2, lr__C=0.1, svd__n_components=120 ................
[CV] lr__penalty=l2, lr__C=0.1, svd__n_components=120 ................
[CV] lr__penalty=l2, lr__C=0.1, svd__n_components=180 ................
[CV] lr__penalty=l2, lr__C=0.1, svd__n_components=180 ................
[CV]  lr__penalty=l1, lr__C=0.1, svd__n_components=120, score=-1.21435731963, total=  18.0s
[CV] lr__penalty=l1, lr__C=1.0, svd__n_components=120 ................
[CV]  lr__penalty=l1, lr__C=0.1, svd__n_components=120, score=-1.2179688342, total=  20.5s
[CV] lr__penalty=l1, lr__C=1.0, svd__n_components=120 ................
[CV]  lr__penalty=l2, lr__C=0.1, svd__n_compon

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   21.3s


[CV] lr__penalty=l1, lr__C=1.0, svd__n_components=180 ................
[CV]  lr__penalty=l2, lr__C=0.1, svd__n_components=120, score=-1.21748807569, total=  20.9s
[CV] lr__penalty=l1, lr__C=1.0, svd__n_components=180 ................
[CV]  lr__penalty=l2, lr__C=0.1, svd__n_components=180, score=-1.11942338388, total=  28.8s
[CV] lr__penalty=l2, lr__C=1.0, svd__n_components=120 ................
[CV]  lr__penalty=l2, lr__C=0.1, svd__n_components=180, score=-1.1192203202, total=  29.5s
[CV] lr__penalty=l2, lr__C=1.0, svd__n_components=120 ................
[CV]  lr__penalty=l1, lr__C=0.1, svd__n_components=180, score=-1.11908777712, total=  31.4s
[CV] lr__penalty=l2, lr__C=1.0, svd__n_components=180 ................
[CV]  lr__penalty=l1, lr__C=0.1, svd__n_components=180, score=-1.11796775141, total=  31.7s
[CV] lr__penalty=l2, lr__C=1.0, svd__n_components=180 ................
[CV]  lr__penalty=l1, lr__C=1.0, svd__n_components=120, score=-1.18973585731, total=  34.8s
[CV] lr__penalty=l1, lr

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   54.2s


[CV]  lr__penalty=l1, lr__C=1.0, svd__n_components=120, score=-1.19737886789, total=  36.5s
[CV] lr__penalty=l1, lr__C=10, svd__n_components=120 .................
[CV]  lr__penalty=l2, lr__C=1.0, svd__n_components=120, score=-1.19154313811, total=  29.5s
[CV] lr__penalty=l1, lr__C=10, svd__n_components=180 .................
[CV]  lr__penalty=l2, lr__C=1.0, svd__n_components=120, score=-1.19691083949, total=  29.2s
[CV] lr__penalty=l1, lr__C=10, svd__n_components=180 .................


...

[CV]  lr__penalty=l2, lr__C=10, svd__n_components=120, score=-1.19675199186, total=  28.1s
[CV]  lr__penalty=l1, lr__C=1.0, svd__n_components=180, score=-1.08731036011, total= 1.4min
[CV]  lr__penalty=l2, lr__C=10, svd__n_components=180, score=-1.09167836891, total=  29.7s


[Parallel(n_jobs=-1)]: Done  21 out of  24 | elapsed:  2.0min remaining:   17.4s


[CV]  lr__penalty=l2, lr__C=10, svd__n_components=180, score=-1.09314138166, total=  25.3s
[CV]  lr__penalty=l1, lr__C=10, svd__n_components=180, score=-1.09056821208, total= 1.1min
[CV]  lr__penalty=l1, lr__C=10, svd__n_components=180, score=-1.09425929022, total= 1.2min


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.2min finished


Best score: -1.086
Best parameters set:
	lr__C: 1.0
	lr__penalty: 'l1'
	svd__n_components: 180


In [67]:
# This technique can be used to finetune xgboost or even multinomial naive bayes as below. We will use the tfidf data here:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, np.asarray(train_data_y))  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] nb__alpha=0.001 .................................................
[CV] nb__alpha=0.001 .................................................
[CV] ............ nb__alpha=0.001, score=-1.22287939967, total=   0.1s
[CV] nb__alpha=0.01 ..................................................
[CV] ............ nb__alpha=0.001, score=-1.21625014609, total=   0.1s


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.3s


[CV] nb__alpha=0.01 ..................................................
[CV] ............. nb__alpha=0.01, score=-1.23879005438, total=   0.1s
[CV] nb__alpha=0.1 ...................................................
[CV] ............. nb__alpha=0.01, score=-1.22638046816, total=   0.1s


[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.6s remaining:    1.9s


[CV] nb__alpha=0.1 ...................................................
[CV] .............. nb__alpha=0.1, score=-1.30567716648, total=   0.1s
[CV] nb__alpha=1 .....................................................
[CV] .............. nb__alpha=0.1, score=-1.29148632123, total=   0.1s


...

[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    1.2s remaining:    0.8s


[CV] nb__alpha=10 ....................................................
[CV] ............... nb__alpha=10, score=-2.90402435607, total=   0.1s
[CV] nb__alpha=100 ...................................................
[CV] ............... nb__alpha=10, score=-2.89400785782, total=   0.1s


[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    1.4s remaining:    0.5s


[CV] nb__alpha=100 ...................................................
[CV] .............. nb__alpha=100, score=-2.71141588116, total=   0.1s
[CV] .............. nb__alpha=100, score=-2.71054921449, total=   0.1s
Best score: -1.220
Best parameters set:
	nb__alpha: 0.001


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.8s finished


In [5]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
# nltk.download()
# stop_words = stopwords.words('english')
from tensorflow.python.client import device_lib

def get_available_devices():  
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

# Word Vectors
# load the GloVe vectors in a dictionary:
from tqdm import tqdm
embeddings_index = {}
f = open('/home/shringi/Downloads/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))



Using TensorFlow backend.
2571it [00:00, 25696.67it/s]

[u'/cpu:0', u'/gpu:0']


2196017it [01:22, 26724.03it/s]

Found 2196016 word vectors.


In [9]:
# this function creates a normalized vector for the whole sentence
from nltk import word_tokenize
def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = tokenize_chars(words)
#     words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(450)
    return v / np.sqrt((v ** 2).sum())

# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(train_data_x)]
xvalid_glove = [sent2vec(x) for x in tqdm(test_data_x)]

xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

print  xtrain_glove[:4]
print  xvalid_glove[:4]

100%|██████████| 6503/6503 [00:01<00:00, 4756.94it/s]

[[-0.0327601   0.07798596 -0.08264437 ..., -0.00731602  0.01921969
   0.04853879]
 [-0.0256834   0.07093357 -0.07748109 ..., -0.01181087  0.02081458
   0.05021318]
 [-0.03204267  0.07462256 -0.08283966 ..., -0.01119007  0.0207957
   0.04974462]
 [-0.03373921  0.06973441 -0.07722378 ..., -0.00863715  0.02233366
   0.05069098]]
[[-0.02916365  0.07641376 -0.08132176 ..., -0.01210064  0.01892132
   0.05505707]
 [-0.02819736  0.07700168 -0.0813869  ..., -0.01140574  0.02110318
   0.04664875]
 [-0.03109495  0.07669184 -0.08212794 ..., -0.00657532  0.020209
   0.05389968]
 [-0.02792529  0.07169788 -0.07927565 ..., -0.009637    0.02120848
   0.05436906]]


In [23]:
# Fitting a xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, train_data_y)
predictions = clf.predict_proba(xvalid_glove)
print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"

logloss: 1.957 
0.141304341394% is the accuracy percentage


In [20]:
# Fitting a simple xgboost on glove features
import xgboost as xgb
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, train_data_y)
predictions = clf.predict_proba(xvalid_glove)
print ("logloss: %0.3f " % metrics.log_loss(np.asarray(test_data_y), predictions))
print str(math.exp(-(metrics.log_loss(test_data_y, predictions)))) + "% is the accuracy percentage"



logloss: 1.946 
0.142898930209% is the accuracy percentage


In [10]:
# Deep Learning application -
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)


# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical((train_data_y))
yvalid_enc = np_utils.to_categorical((test_data_y))

# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(12))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=350, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 26010 samples, validate on 6503 samples
Epoch 1/350
26010/26010 [==============================] - 1s - loss: 2.2300 - val_loss: 1.9343
Epoch 2/350
26010/26010 [==============================] - 1s - loss: 1.9497 - val_loss: 1.8554
Epoch 3/350
26010/26010 [==============================] - 1s - loss: 1.8701 - val_loss: 1.8322
Epoch 4/350
26010/26010 [==============================] - 1s - loss: 1.8259 - val_loss: 1.8097
Epoch 5/350
26010/26010 [==============================] - 1s - loss: 1.8003 - val_loss: 1.7814
Epoch 6/350
26010/26010 [==============================] - 1s - loss: 1.7854 - val_loss: 1.7727
Epoch 7/350
26010/26010 [==============================] - 1s - loss: 1.7723 - val_loss: 1.7686
Epoch 8/350
26010/26010 [==============================] - 1s - loss: 1.7531 - val_loss: 1.7635
Epoch 9/350
26010/26010 [==============================] - 1s - loss: 1.7466 - val_loss: 1.7579
Epoch 10/350
26010/26010 [==============================] - 1s - loss: 1.7368 - val_los

26010/26010 [==============================] - 1s - loss: 1.4867 - val_loss: 1.7537
Epoch 86/350
26010/26010 [==============================] - 1s - loss: 1.4914 - val_loss: 1.7581
Epoch 87/350
26010/26010 [==============================] - 1s - loss: 1.4810 - val_loss: 1.7574
Epoch 88/350
26010/26010 [==============================] - 1s - loss: 1.4850 - val_loss: 1.7556
Epoch 89/350
26010/26010 [==============================] - 1s - loss: 1.4829 - val_loss: 1.7637
Epoch 90/350
26010/26010 [==============================] - 1s - loss: 1.4816 - val_loss: 1.7620
Epoch 91/350
26010/26010 [==============================] - 1s - loss: 1.4761 - val_loss: 1.7556
Epoch 92/350
26010/26010 [==============================] - 1s - loss: 1.4837 - val_loss: 1.7564
Epoch 93/350
26010/26010 [==============================] - 1s - loss: 1.4731 - val_loss: 1.7524
Epoch 94/350
26010/26010 [==============================] - 1s - loss: 1.4717 - val_loss: 1.7555
Epoch 95/350
26010/26010 [=================

26010/26010 [==============================] - 1s - loss: 1.3441 - val_loss: 1.8167
Epoch 252/350
26010/26010 [==============================] - 1s - loss: 1.3499 - val_loss: 1.8105
Epoch 253/350
26010/26010 [==============================] - 1s - loss: 1.3501 - val_loss: 1.8127
Epoch 254/350
26010/26010 [==============================] - 1s - loss: 1.3511 - val_loss: 1.8144
Epoch 255/350
26010/26010 [==============================] - 1s - loss: 1.3433 - val_loss: 1.8116
Epoch 256/350
26010/26010 [==============================] - 1s - loss: 1.3485 - val_loss: 1.8078
Epoch 257/350
26010/26010 [==============================] - 1s - loss: 1.3429 - val_loss: 1.8049
Epoch 258/350
26010/26010 [==============================] - 1s - loss: 1.3444 - val_loss: 1.8132
Epoch 259/350
26010/26010 [==============================] - 1s - loss: 1.3447 - val_loss: 1.8093
Epoch 260/350
26010/26010 [==============================] - 1s - loss: 1.3417 - val_loss: 1.8115
Epoch 261/350
26010/26010 [=======

In [11]:
# print type(ytrain_enc)
# print ytrain_enc.shape
# print yvalid_enc.shape
print xtrain_glove_scl.shape
# print xtrain_glove_scl[0]
# print xtrain_glove[0]
print len(xtrain_glove[45])



(26010, 300)
300


In [12]:
#USING LSTM's now -----
# using keras tokenizer here
token = text.Tokenizer(num_words=None, char_level=True)
max_len = 450

token.fit_on_texts(list(train_data_x) + list(test_data_x))
xtrain_seq = token.texts_to_sequences(train_data_x)
xvalid_seq = token.texts_to_sequences(test_data_x)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [13]:
print len(xtrain_seq)
print len(xvalid_seq)

print len(xvalid_pad[0])

print xtrain_pad[0]
print word_index

26010
6503
450
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 15 11  8  7 11  3
  1  6  1  2 13 17  1  6 11  3  6 16  3  4  5 18  1  2 13  4 12 16 10  3  6
  5  8  7  1  6 23  9  8  7 15 12  1  2  6  5  3  4  8  7 11  3  1  6  1  2
  9  7  9  7  5 14  1  2  6 16  4  9  3  4 13  4 12 16  1  2  1  6  5 14 11
  3  1  6  1  2 13 17 13  4  8  7  4  9  3  4  9  7  1  2  8  7 11  3  1  6
  1  2  1  6 13  4 12 16  5 14 11  3 23  9  8  7  1  2  4  9 10  3 10  3 13
  4  1  2 15 12  9  7 12 16 10  3 19 20  1  2  6 16  4  9  3  4 15 12  4  9
 12 16 10  3 13  4  1  2 15 12  3  4 17 18  5 14  6  5  1  2 13  4  3  4 11
  3  1  6  1  2  1  6  5 14 11  3  1  6  1  2 13  4  6  5  8  7  8  7  4  9
 10  3 10  3 17 18  6  5  5 14 11  3  1  2  8  7 11  3  1  6  1  2  8  7  6
  5  8  7 19 20  1  2  9  7 12 16 22  5  6  5  5 14  8  7 22  5  1  2 10  3
 13  4  1  2  8  7  9  7  3  4 11  3 19 20  1  2 10 12  5 18  8  7 11  3 

In [64]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical((train_data_y))
yvalid_enc = np_utils.to_categorical((test_data_y))        

# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(12))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=500, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc))

100%|██████████| 26/26 [00:00<00:00, 170927.75it/s]


Train on 26010 samples, validate on 6503 samples
Epoch 1/500
26010/26010 [==============================] - 21s - loss: 2.3735 - acc: 0.1413 - val_loss: 2.3552 - val_acc: 0.1610
Epoch 2/500
26010/26010 [==============================] - 21s - loss: 2.3163 - acc: 0.1678 - val_loss: 2.2980 - val_acc: 0.1827
Epoch 3/500
26010/26010 [==============================] - 21s - loss: 2.2778 - acc: 0.1814 - val_loss: 2.2620 - val_acc: 0.1856
Epoch 4/500
26010/26010 [==============================] - 21s - loss: 2.2538 - acc: 0.1906 - val_loss: 2.2444 - val_acc: 0.1925
Epoch 5/500
26010/26010 [==============================] - 21s - loss: 2.2366 - acc: 0.1989 - val_loss: 2.2227 - val_acc: 0.1970
Epoch 6/500
26010/26010 [==============================] - 21s - loss: 2.2215 - acc: 0.1989 - val_loss: 2.2140 - val_acc: 0.2022
Epoch 7/500
26010/26010 [==============================] - 21s - loss: 2.2112 - acc: 0.2016 - val_loss: 2.2066 - val_acc: 0.2079
Epoch 8/500
26010/26010 [=======================

26010/26010 [==============================] - 20s - loss: 1.9774 - acc: 0.3059 - val_loss: 1.9497 - val_acc: 0.3262
Epoch 64/500
26010/26010 [==============================] - 21s - loss: 1.9742 - acc: 0.3107 - val_loss: 1.9513 - val_acc: 0.3278
Epoch 65/500
26010/26010 [==============================] - 20s - loss: 1.9687 - acc: 0.3120 - val_loss: 1.9409 - val_acc: 0.3254
Epoch 66/500
26010/26010 [==============================] - 21s - loss: 1.9674 - acc: 0.3120 - val_loss: 1.9324 - val_acc: 0.3314
Epoch 67/500
26010/26010 [==============================] - 21s - loss: 1.9549 - acc: 0.3191 - val_loss: 1.9248 - val_acc: 0.3365
Epoch 68/500
26010/26010 [==============================] - 21s - loss: 1.9495 - acc: 0.3193 - val_loss: 1.9155 - val_acc: 0.3392
Epoch 69/500
26010/26010 [==============================] - 20s - loss: 1.9436 - acc: 0.3200 - val_loss: 1.8916 - val_acc: 0.3441
Epoch 70/500
26010/26010 [==============================] - 21s - loss: 1.9275 - acc: 0.3256 - val_loss

26010/26010 [==============================] - 21s - loss: 1.3634 - acc: 0.5188 - val_loss: 1.3206 - val_acc: 0.5410
Epoch 126/500
26010/26010 [==============================] - 21s - loss: 1.3555 - acc: 0.5230 - val_loss: 1.3145 - val_acc: 0.5373
Epoch 127/500
26010/26010 [==============================] - 21s - loss: 1.3477 - acc: 0.5251 - val_loss: 1.3094 - val_acc: 0.5401
Epoch 128/500
26010/26010 [==============================] - 21s - loss: 1.3327 - acc: 0.5236 - val_loss: 1.2872 - val_acc: 0.5530
Epoch 129/500
26010/26010 [==============================] - 21s - loss: 1.3305 - acc: 0.5303 - val_loss: 1.2921 - val_acc: 0.5441
Epoch 130/500
26010/26010 [==============================] - 21s - loss: 1.3194 - acc: 0.5302 - val_loss: 1.2806 - val_acc: 0.5467
Epoch 131/500
26010/26010 [==============================] - 21s - loss: 1.3224 - acc: 0.5288 - val_loss: 1.2964 - val_acc: 0.5424
Epoch 132/500
26010/26010 [==============================] - 21s - loss: 1.3032 - acc: 0.5373 - v

26010/26010 [==============================] - 21s - loss: 1.1520 - acc: 0.5826 - val_loss: 1.1294 - val_acc: 0.5900
Epoch 188/500
26010/26010 [==============================] - 21s - loss: 1.1469 - acc: 0.5868 - val_loss: 1.1265 - val_acc: 0.5885
Epoch 189/500
26010/26010 [==============================] - 21s - loss: 1.1481 - acc: 0.5860 - val_loss: 1.1260 - val_acc: 0.5882
Epoch 190/500
26010/26010 [==============================] - 21s - loss: 1.1429 - acc: 0.5856 - val_loss: 1.1247 - val_acc: 0.5943
Epoch 191/500
26010/26010 [==============================] - 21s - loss: 1.1420 - acc: 0.5846 - val_loss: 1.1332 - val_acc: 0.5960
Epoch 192/500
26010/26010 [==============================] - 21s - loss: 1.1400 - acc: 0.5879 - val_loss: 1.1220 - val_acc: 0.5974
Epoch 193/500
26010/26010 [==============================] - 21s - loss: 1.1334 - acc: 0.5901 - val_loss: 1.1254 - val_acc: 0.5933
Epoch 194/500
26010/26010 [==============================] - 21s - loss: 1.1357 - acc: 0.5917 - v

26010/26010 [==============================] - 21s - loss: 1.0721 - acc: 0.6148 - val_loss: 1.0672 - val_acc: 0.6162
Epoch 250/500
26010/26010 [==============================] - 21s - loss: 1.0637 - acc: 0.6163 - val_loss: 1.0625 - val_acc: 0.6183
Epoch 251/500
26010/26010 [==============================] - 21s - loss: 1.0613 - acc: 0.6195 - val_loss: 1.0649 - val_acc: 0.6182
Epoch 252/500
26010/26010 [==============================] - 21s - loss: 1.0681 - acc: 0.6159 - val_loss: 1.0663 - val_acc: 0.6160
Epoch 253/500
26010/26010 [==============================] - 21s - loss: 1.0614 - acc: 0.6180 - val_loss: 1.0616 - val_acc: 0.6211
Epoch 254/500
26010/26010 [==============================] - 21s - loss: 1.0603 - acc: 0.6220 - val_loss: 1.0705 - val_acc: 0.6194
Epoch 255/500
26010/26010 [==============================] - 21s - loss: 1.0539 - acc: 0.6234 - val_loss: 1.0612 - val_acc: 0.6202
Epoch 256/500
26010/26010 [==============================] - 21s - loss: 1.0627 - acc: 0.6193 - v

26010/26010 [==============================] - 21s - loss: 1.0194 - acc: 0.6379 - val_loss: 1.0210 - val_acc: 0.6322
Epoch 312/500
26010/26010 [==============================] - 21s - loss: 1.0049 - acc: 0.6402 - val_loss: 1.0112 - val_acc: 0.6402
Epoch 313/500
 2560/26010 [=>............................] - ETA: 17s - loss: 1.0077 - acc: 0.6473

KeyboardInterrupt: 

In [67]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical((train_data_y))
yvalid_enc = np_utils.to_categorical((test_data_y))

embedding_matrix.shape
# embedding_matrix[0]



100%|██████████| 26/26 [00:00<00:00, 199728.76it/s]


(27, 300)

In [16]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(12))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=64, epochs=200, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 26010 samples, validate on 6503 samples
Epoch 1/200
26010/26010 [==============================] - 305s - loss: 2.3224 - val_loss: 2.2337
Epoch 2/200
26010/26010 [==============================] - 369s - loss: 2.2166 - val_loss: 2.1770
Epoch 3/200
 6016/26010 [=====>........................] - ETA: 230s - loss: 2.2030

KeyboardInterrupt: 

In [18]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(12))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 26010 samples, validate on 6503 samples
Epoch 1/100
26010/26010 [==============================] - 230s - loss: 2.3901 - val_loss: 2.3528
Epoch 2/100
26010/26010 [==============================] - 231s - loss: 2.3114 - val_loss: 2.2849
Epoch 3/100
26010/26010 [==============================] - 230s - loss: 2.2724 - val_loss: 2.1916
Epoch 4/100
26010/26010 [==============================] - 230s - loss: 2.1454 - val_loss: 2.0725
Epoch 5/100
26010/26010 [==============================] - 230s - loss: 2.0896 - val_loss: 2.0362
Epoch 6/100
26010/26010 [==============================] - 229s - loss: 2.0637 - val_loss: 2.0301
Epoch 7/100
26010/26010 [==============================] - 230s - loss: 2.0619 - val_loss: 2.0359
Epoch 8/100
26010/26010 [==============================] - 230s - loss: 2.0557 - val_loss: 2.0349
Epoch 9/100
26010/26010 [==============================] - 231s - loss: 2.0554 - val_loss: 2.0332
Epoch 10/100
26010/26010 [==============================] - 231s - lo

In [29]:
# Training a 1D convnet with existing GloVe features/vectors
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(128,
                 5,
                 activation='relu'))
# we use max pooling:
model.add(MaxPooling1D(5))

model.add(Conv1D(128,
                 5,
                 activation='relu'))
# we use max pooling:
model.add(MaxPooling1D(5))

model.add(Conv1D(128,
                 5,
                 activation='relu'))
# we use max pooling:
model.add(MaxPooling1D(3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(12, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])


# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc))



Train on 26010 samples, validate on 6503 samples
Epoch 1/100
26010/26010 [==============================] - 5s - loss: 2.3156 - acc: 0.1661 - val_loss: 2.1369 - val_acc: 0.2288
Epoch 2/100
26010/26010 [==============================] - 4s - loss: 1.9449 - acc: 0.3203 - val_loss: 1.8059 - val_acc: 0.3791
Epoch 3/100
26010/26010 [==============================] - 4s - loss: 1.5747 - acc: 0.4494 - val_loss: 1.4401 - val_acc: 0.4925
Epoch 4/100
26010/26010 [==============================] - 4s - loss: 1.2503 - acc: 0.5613 - val_loss: 1.2287 - val_acc: 0.5619
Epoch 5/100
26010/26010 [==============================] - 4s - loss: 1.0488 - acc: 0.6328 - val_loss: 1.1942 - val_acc: 0.5911
Epoch 6/100
26010/26010 [==============================] - 4s - loss: 0.8775 - acc: 0.6954 - val_loss: 1.2926 - val_acc: 0.5763
Epoch 7/100
26010/26010 [==============================] - 4s - loss: 0.7346 - acc: 0.7488 - val_loss: 1.2406 - val_acc: 0.6065
Epoch 8/100
26010/26010 [==============================

In [54]:
#Preparing test data for final predictions

lines_pred = [line.rstrip('\n') for line in open("/home/shringi/Downloads/offline_challenge/offline_challenge/xtest_obfuscated.txt")]
# with open("/home/dupree/Downloads/offline_challenge/offline_challenge/xtrain_obfuscated.txt") as f:
#     lines = f.readlines()
# random.shuffle(lines)
pred_data_x = lines_pred
len(pred_data_x)
xpred_seq = token.texts_to_sequences(pred_data_x)

# zero pad the sequences
xpred_pad = sequence.pad_sequences(xpred_seq, maxlen=max_len)

print len(xpred_pad)
# print xpred_pad[0]
# result = model.predict(xpred_pad)
# print result[:5]
predictions = model.predict_classes(xpred_pad)
print predictions[:5]
print (predictions).shape

for class_label in predictions:
    print class_label

3000
2464/3000 [=======================>......] - ETA: 0s[6 1 8 8 4]
(3000,)
6
1
8
8
4
10
6
6
10
8
1
7
5
10
7
10
1
6
3
8
10
11
5
6
9
7
8
2
6
6
7
6
1
5
8
3
10
8
1
8
6
6
9
3
1
11
1
5
7
8
7
3
7
6
8
5
9
7
11
7
4
8
7
4
7
3
7
6
10
3
7
4
10
6
6
7
4
6
7
8
8
6
1
6
1
5
7
1
8
5
1
10
3
8
7
8
1
6
6
3
6
8
8
9
8
11
1
3
5
6
6
1
3
3
7
1
4
1
7
10
5
5
1
7
1
3
1
3
1
7
6
3
7
6
6
6
8
7
2
8
1
10
7
1
2
3
4
7
10
8
6
3
3
3
7
3
4
1
3
8
3
6
7
3
7
8
6
7
10
8
5
6
7
10
6
6
7
2
10
1
9
11
6
7
1
0
8
11
7
1
3
1
3
1
1
7
6
5
7
7
6
1
1
7
3
3
1
7
8
3
8
5
7
9
1
3
6
10
7
3
6
5
4
6
6
6
2
4
3
10
1
7
7
6
7
3
2
3
11
8
4
8
2
6
9
7
7
1
10
1
6
3
1
4
7
6
3
3
3
4
7
1
10
8
4
10
7
8
7
7
8
5
8
3
0
7
1
7
3
1
1
6
3
6
1
6
8
7
4
3
7
0
7
1
6
1
5
10
6
9
9
3
5
4
2
7
8
10
10
5
10
7
4
11
4
8
5
1
2
10
10
8
0
8
4
4
8
5
1
7
7
7
6
11
5
6
6
6
7
7
7
0
11
6
3
1
7
7
7
7
7
7
1
7
6
3
7
4
6
8
8
7
10
7
7
7
8
4
5
7
7
4
1
8
6
1
6
1
6
3
3
8
1
1
8
6
3
3
7
1
9
10
3
8
1
7
2
6
7
10
6
5
8
6
6
7
1
8
2
7
7
3
10
8
10
4
1
8
7
3
7
6
1
1
7
0
3
7
10
2
3
7
3
7
2
8
10
1
10
1

In [59]:
# Using a Convolution1D, with GloVe features
# # scale the data before any neural net:
# scl = preprocessing.StandardScaler()
# xtrain_glove_scl = scl.fit_transform(xtrain_glove)
# xvalid_glove_scl = scl.transform(xvalid_glove)

# model.add(Dense(300, input_dim=300, activation='relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

# model.add(Dense(300, activation='relu'))
# model.add(Dropout(0.3))
# model.add(BatchNormalization())

# model.add(Dense(12))
# model.add(Activation('softmax'))

# Training a 1D convnet with existing GloVe features/vectors
model = Sequential()

model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(Dropout(0.2))
# model.add(BatchNormalization())

# word group filters of size filter_length:
model.add(Conv1D(128,
                 5,
                 activation='relu'))
# we use max pooling:
model.add(MaxPooling1D(5))

model.add(Conv1D(128,
                 5,
                 activation='relu'))
# we use max pooling:
model.add(MaxPooling1D(5))

model.add(Conv1D(128,
                 5,
                 activation='relu'))
# we use max pooling:
model.add(MaxPooling1D(3))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(12, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
# model.fit(x_train, y_train, validation_data=(x_val, y_val),
#           epochs=2, batch_size=128)

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=64, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc))




Train on 26010 samples, validate on 6503 samples
Epoch 1/100
26010/26010 [==============================] - 4s - loss: 2.2635 - acc: 0.1896 - val_loss: 2.1370 - val_acc: 0.2470
Epoch 2/100
26010/26010 [==============================] - 4s - loss: 1.8503 - acc: 0.3499 - val_loss: 1.5993 - val_acc: 0.4215
Epoch 3/100
 2752/26010 [==>...........................] - ETA: 3s - loss: 1.5537 - acc: 0.4371

KeyboardInterrupt: 